In [18]:
from json_processor import build_data_set
from constant import NEW_LOG, OLD_LOG, VERSION_INPUT, VERSION_OUTPUT
from common import translate_html_entity
import pandas as pd
import os
import logging
import json
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/pkun/.ssh/payuat-985b0c12a3d4.json"
os.chdir('/Users/pkun/PycharmProjects/ui_api_automated_test')
LOG = logging.getLogger('data_build')
chinese_map_english = json.load(open('data/chinese_map_english.json', 'r'))

In [2]:
def translate_seria_text_to_english(seria):
    import six
    from google.cloud import translate_v2 as translate

    translate_client = translate.Client()
    result = dict() 
    # Text can also be a sequence of strings, in which case this method
    # will return a sequence of results for each text.
    count = 0
    for data in seria:
        result[data] = translate_client.translate(
            data, target_language='en')['translatedText']
        result[data] = translate_html_entity(result[data])
        count += 1
        if count % 200 == 0:
            print(count)
            print(len(result))
    return result

In [3]:
def translate_single_text_to_english(text):
    return chinese_map_english[text]


In [3]:
# train_indices = np.random.choice(8052, int(8052*0.8), replace=False)
# test_indices = np.delete(np.arange(8052), train_indices)
# train_data = copy_data_set[train_indices]
# test_data = copy_data_set[test_indices] 

In [4]:
def from_data_to_tsv(data_set, file):
    f = open(file, 'w')
    for i in data_set:
        f.write(i.to_text()+'\n')
    f.close()

In [5]:
new_log_data_set = build_data_set(NEW_LOG)
old_log_data_set = build_data_set(OLD_LOG)

In [6]:
new_log_data_set

,label,query,ui_info
0,yes,confirm payment information to confirm payment...,立即支付
1,no,confirm payment information to confirm payment...,"当前所在页面,支付"
2,no,confirm payment information to confirm payment...,返回
3,no,confirm payment information to confirm payment...,支付
4,no,confirm payment information to confirm payment...,body
...,...,...,...
194733,no,user to submit cardholder information when fil...,选择
194734,no,user to submit cardholder information when fil...,同意用户服务协议
194735,no,user to submit cardholder information when fil...,下一步
194736,no,user to submit cardholder information when fil...,将发起0.05美元预授权以验证账户


In [8]:
def remove_duplicates_with_label_eq_no(data_set):
    data_set[data_set['label']=='no'] = data_set[data_set['label']=='no'].drop_duplicates(['query', 'ui_info'])
    data_set.dropna(axis=0, how='any', inplace=True)

In [9]:
remove_duplicates_with_label_eq_no(new_log_data_set)
remove_duplicates_with_label_eq_no(old_log_data_set)
old_log_data_set

,label,query,ui_info
0,yes,confirm payment information to confirm payment...,立即支付
1,no,confirm payment information to confirm payment...,"当前所在页面,支付"
2,no,confirm payment information to confirm payment...,支付
3,no,confirm payment information to confirm payment...,返回
4,no,confirm payment information to confirm payment...,body
...,...,...,...
16340,yes,confirm payment information to confirm payment...,立即支付
16349,yes,confirm payment information to confirm payment...,立即支付
16358,yes,confirm payment information to confirm payment...,立即支付
16367,yes,submit payment password authorized payment vie...,支付方式


In [10]:
def describe_label_number(data_set):
    print(data_set.head())
    print('Yes', len(data_set[data_set['label'] == 'yes']))
    print('No', len(data_set[data_set['label'] == 'no']))
    data_set

In [11]:
describe_label_number(new_log_data_set)
describe_label_number(old_log_data_set)

  label                                              query    ui_info
0   yes  confirm payment information to confirm payment...       立即支付
1    no  confirm payment information to confirm payment...  当前所在页面,支付
2    no  confirm payment information to confirm payment...         返回
3    no  confirm payment information to confirm payment...         支付
4    no  confirm payment information to confirm payment...       body
Yes 7450
No 7297
  label                                              query    ui_info
0   yes  confirm payment information to confirm payment...       立即支付
1    no  confirm payment information to confirm payment...  当前所在页面,支付
2    no  confirm payment information to confirm payment...         支付
3    no  confirm payment information to confirm payment...         返回
4    no  confirm payment information to confirm payment...       body
Yes 796
No 877


In [12]:
new_log_data_set['ui_info'].head().apply(translate_single_text_to_english)

0      pay immediately
1    Current page, pay
2               return
3               To pay
4                 body
Name: ui_info, dtype: object

In [13]:
new_log_data_set['ui_info'] = new_log_data_set['ui_info'].apply(translate_single_text_to_english)
old_log_data_set['ui_info'] = old_log_data_set['ui_info'].apply(translate_single_text_to_english)

In [19]:
new_log_data_set.to_csv(path_or_buf=VERSION_INPUT, sep='\t', header=False, index=False)
old_log_data_set.to_csv(path_or_buf=VERSION_OUTPUT, sep='\t', header=False, index=False)

In [ ]:
input_data_set = new_log_data_set
